In [ ]:
import chromadb
from langchain_huggingface import HuggingFaceEmbeddings
from chromadb import Documents, EmbeddingFunction, Embeddings

In [ ]:
chroma_client = chromadb.PersistentClient(path="./chroma_data")

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B")

In [ ]:
class CustomEmbeddingFunction(EmbeddingFunction):
    def __init__(self, embedding_model: HuggingFaceEmbeddings):
        self.embedding_model = embedding_model

    def __call__(self, texts: Documents) -> Embeddings:
        return self.embedding_model.embed_documents(texts)

In [ ]:
chroma_collection = chroma_client.get_or_create_collection(
    name="requirements_collection_qwen3",
    embedding_function=CustomEmbeddingFunction(embedding_model=embedding_model),
)